<a href="https://colab.research.google.com/github/Akshithkumarreddy/Machine_learning_AL/blob/main/%23001_CNN_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
!tar -xvf cifar-100-python.tar.gz

--2025-05-18 12:01:29--  https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169001437 (161M) [application/x-gzip]
Saving to: ‘cifar-100-python.tar.gz’

cifar-100-python.ta 100%[===================>] 161.17M  56.4MB/s    in 2.9s    

2025-05-18 12:01:32 (56.4 MB/s) - ‘cifar-100-python.tar.gz’ saved [169001437/169001437]

cifar-100-python/
cifar-100-python/file.txt~
cifar-100-python/train
cifar-100-python/test
cifar-100-python/meta


In [2]:
from matplotlib import pyplot as plt
import numpy as np
import os
from tqdm import tqdm

In [3]:
def unpickle(file):  #Pickling - where a python object hierarchy is converted into a byte stream ; Unpickling - byte stream converted into a object hierarchy
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

train_data = unpickle('cifar-100-python/train')
test_data = unpickle('cifar-100-python/test')
meta = unpickle('cifar-100-python/meta')

print(train_data.keys())
print(train_data[b'data'].shape)
print(test_data.keys())
print(test_data[b'data'].shape)

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])
(50000, 3072)
dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])
(10000, 3072)


In [4]:
labels = meta[b'fine_label_names']
print(labels)

[b'apple', b'aquarium_fish', b'baby', b'bear', b'beaver', b'bed', b'bee', b'beetle', b'bicycle', b'bottle', b'bowl', b'boy', b'bridge', b'bus', b'butterfly', b'camel', b'can', b'castle', b'caterpillar', b'cattle', b'chair', b'chimpanzee', b'clock', b'cloud', b'cockroach', b'couch', b'crab', b'crocodile', b'cup', b'dinosaur', b'dolphin', b'elephant', b'flatfish', b'forest', b'fox', b'girl', b'hamster', b'house', b'kangaroo', b'keyboard', b'lamp', b'lawn_mower', b'leopard', b'lion', b'lizard', b'lobster', b'man', b'maple_tree', b'motorcycle', b'mountain', b'mouse', b'mushroom', b'oak_tree', b'orange', b'orchid', b'otter', b'palm_tree', b'pear', b'pickup_truck', b'pine_tree', b'plain', b'plate', b'poppy', b'porcupine', b'possum', b'rabbit', b'raccoon', b'ray', b'road', b'rocket', b'rose', b'sea', b'seal', b'shark', b'shrew', b'skunk', b'skyscraper', b'snail', b'snake', b'spider', b'squirrel', b'streetcar', b'sunflower', b'sweet_pepper', b'table', b'tank', b'telephone', b'television', b'ti

In [7]:
dataset_folder = "/content/cifar_100_images/train"
os.makedirs(dataset_folder, exist_ok=True)
for c in tqdm(range(len(train_data[b'data']))):  #tqdm is a Python library that wraps around an iterable (like range(...)) to show a progress bar in the terminal or notebook
    data = train_data[b'data'][c]
    label = train_data[b'fine_labels'][c]
    r, g, b = data[0:1024].reshape(32, 32), data[1024:2048].reshape(32, 32), data[2048:].reshape(32, 32)
    img = np.dstack((r, g, b))
    plt.imsave(f"{dataset_folder}/{labels[label]}_{c}.png", img)

dataset_folder = "/content/cifar_100_images/test"
os.makedirs(dataset_folder, exist_ok=True)
for c in tqdm(range(len(test_data[b'data']))):
    data = test_data[b'data'][c]
    label = test_data[b'fine_labels'][c]
    r, g, b = data[0:1024].reshape(32, 32), data[1024:2048].reshape(32, 32), data[2048:].reshape(32, 32)
    img = np.dstack((r, g, b))
    plt.imsave(f"{dataset_folder}/{labels[label]}_{c}.png", img)

100%|██████████| 10000/10000 [00:08<00:00, 1180.91it/s]


In [8]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder

class CustomImageDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset[idx]
        img = plt.imread(img_path)
        img = np.transpose(img[:,:,:3], (2, 0, 1))
        return img, label

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        #Input: B x 3 X 32 X 32
        #Output: B X 4 X 16 X 16
        self.conv1 = nn.Conv2d(3, 4, 3, padding=1, stride=2)

        #Input: B x 4 X 16 X 16
        #Output: B X 8 X 8 X 8
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1, stride=2)

        #Input: B x 8 X 8 X 8
        #Output: B X 16 X 4 X 4
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1, stride=2)

        #Input: B x 16 X 4 X 4
        #Output: B X 32 X 2 X 2
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1, stride=2)

        #Input: B x 32 X 2 X 2
        #Output: B X 64 X 1 X 1
        self.conv5 = nn.Conv2d(32, 64, 3, padding=1, stride=2)

        #Input: B x 64 X 1 X 1
        #Output: B X 100
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 100)


    def forward(self, x):
        print("Input Data: ", x.shape)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

def train_epoch(train_dataloader, model):
    for x, y in tqdm(train_dataloader):
        output = model.forward(x)
        print(x.shape,y, output.shape)

def test_epoch(test_dataloader):
    pass

def train_test(train_dataloader, test_dataloader, model):
    train_epoch(train_dataloader, model)

def main():

    data_folder = "cifar_100_images"
    train_folder = os.path.join(data_folder, "train")
    test_folder = os.path.join(data_folder, "test")

    train_x = [os.path.join(train_folder,f) for f in os.listdir(train_folder)]
    test_x = [os.path.join(test_folder,f) for f in os.listdir(train_folder)]

    train_labels = [f.split('_')[0] for f in os.listdir(train_folder)]
    test_labels = [f.split('_')[0] for f in os.listdir(train_folder)]

    le = LabelEncoder()
    train_labels = le.fit_transform(train_labels)
    test_labels = le.transform(test_labels)

    train_data = [(train_x[i], train_labels[i])for i in range(len(train_x))]
    test_data = [(test_x[i], test_labels[i])for i in range(len(train_x))]

    train_dataset = CustomImageDataset(train_data)
    test_dataset = CustomImageDataset(test_data)

    train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

    model = CNN()

    train_test(train_dataloader, test_dataloader, model)

main()

  0%|          | 0/6250 [00:00<?, ?it/s]

Input Data:  torch.Size([8, 3, 32, 32])


  0%|          | 12/6250 [00:00<02:49, 36.73it/s]

torch.Size([8, 3, 32, 32]) tensor([ 0, 29, 15,  5, 96, 84,  6, 39]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([94, 93, 42, 61, 39, 53, 89, 70]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([23, 53, 32, 52, 58, 14, 59, 84]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([95, 97, 65, 33, 32, 45, 94, 75]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([20, 56, 12,  5, 30, 98, 36,  4]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([94, 40, 77, 27, 68, 82, 45, 19]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([49, 21, 21, 24, 24, 92, 23, 94]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([90, 76, 43,  8, 48, 65, 19, 90]) torch.Size([8, 100])
Input Da

  1%|          | 38/6250 [00:00<01:12, 85.11it/s]

...

Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([ 2, 77, 60, 55, 78, 31, 73, 36]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([10, 34,  2, 76,  6, 60, 14, 54]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([29, 23, 42, 62, 19, 38, 93, 11]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([97, 18, 58, 56,  4, 98, 65,  8]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([17, 48, 93, 81, 69, 62, 87,  7]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([17, 41, 81, 22, 64, 40, 63,  9]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([71, 27, 68, 78,  2, 84, 77,  1]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([97, 90, 13, 74, 50, 12

100%|█████████▉| 6236/6250 [00:40<00:00, 125.88it/s]

Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([76, 67, 80, 44, 23, 79, 21, 10]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([83, 58, 61, 55, 96, 72, 92, 41]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([36, 28, 31, 41, 35, 65, 36, 40]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([57, 80, 13, 68, 57, 72, 27, 17]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([74, 33, 97,  7, 46, 65, 70, 51]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([24, 53, 59, 88, 21, 46,  1,  7]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([75, 52, 94, 12, 60, 65, 85,  2]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([28, 63, 49, 85, 53, 38

100%|██████████| 6250/6250 [00:40<00:00, 155.10it/s]

Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([43, 49,  5,  7, 67, 11, 67, 59]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([59, 72,  9, 59, 79, 67, 82, 88]) torch.Size([8, 100])
Input Data:  torch.Size([8, 3, 32, 32])
torch.Size([8, 3, 32, 32]) tensor([89, 55, 26, 59, 88, 26, 44, 89]) torch.Size([8, 100])
